In [1]:
"""
Purpose; 
To rerun the dendritic width and length calculations on the run 2 pinky data

OBSERVATIONS: 
Had to limit the width to be below 1000 or else had 4 significant outliers
Have not rerun the stats yet


"""

'\nPurpose; \nTo rerun the dendritic width and length calculations on the run 2 pinky data\n\nOBSERVATIONS: \nHad to limit the width to be below 1000 or else had 4 significant outliers\nHave not rerun the stats yet\n\n\n'

In [2]:
import numpy as np
import datajoint as dj
import time
import os
import datetime

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv
import pandas as pd
from tqdm import tqdm



#for filtering
import math
from pykdtree.kdtree import KDTree

import seaborn as sns

In [3]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
ta3p100 = dj.create_virtual_module('pinky', 'microns_pinky')


Connecting celiib@10.28.0.34:3306


NameError: name 'pinky' is not defined

In [ ]:
#lose 10 dendrite segments to not having matching skeleton
len(ta3p100.DendriteWidth()),len(ta3p100.DendriteWidth() & "min_width_max>0")

In [ ]:
ta3p100.DendriteWidth & "min_width_max<0"

In [ ]:
dendrite_accepted = ta3p100.DendriteWidth() & "min_width_max>0"

In [ ]:
#get the distribution of all the 95 %
#pull down the 95 percentile
width_95_all = dendrite_accepted.fetch("min_width_perc_95")
#sns.set()
width_95_all



In [ ]:
dendrite_accepted & "min_width_perc_95>10000"

In [ ]:
#BECAUSE THE COMPONENT INDEX ISN'T ALWAYS THE LARGEST 
#get the list of the largest dendrite components and filter away the smaller ones
"""
Iterate over all dendrites in orphan
1) pull down the segment_ids
2) make unique segments ids (turn into set)
3) For each unique segment id pull down all component index and n_vertex_indices Compartment Rows from the start table:
a. Get the index of the highest vertex count
b. Save the segment_id, component_index and n_vertex_indices as a dict
c. Add it to the list of dicts

Use the list of dicts to restrict the width table

"""

start_table = (ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'").proj("n_vertex_indices")


In [ ]:
start_table = (ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'").proj("n_vertex_indices")
seg_ids = start_table.fetch("segment_id")
len(seg_ids),len(set(seg_ids))

component_total = []
for s in set(seg_ids):
    print(s)
    seg_data = (start_table & "segment_id="+str(s)).fetch("component_index","n_vertex_indices")
    print(seg_data)
    components = seg_data[0]
    n_vertices = seg_data[1]
    print(components)
    print(n_vertices)
    
    #get the maximum n_vertices 
    n_verts_max = max(n_vertices)
    components_max = components[n_vertices == n_verts_max][0]
    
    print(n_verts_max)
    print(components_max)
    
    new_dict = dict(segment_id=s,component_index=components_max,n_vertex_indices=n_verts_max)
    component_total.append(new_dict)



In [ ]:
len(seg_ids),len(set(seg_ids))

In [ ]:
#restrict starter table by dicts to make sure worked
start_table & component_total & "component_index>0"

In [ ]:
id = 648518346341373109
ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'" & "segment_id="+str(id)

In [ ]:
#only want to do orphan width
dendrite_accepted & component_total & "compartment_type='Dendrite'" & "min_width_perc_95>800"

# Beginning of Analysis

In [ ]:
print(len(ta3p100.DendriteWidth() & "min_width_max>0"))
print(len(ta3p100.DendriteWidth() & "min_width_max>0" & ta3p100.CompartmentOrphan.ComponentOrphan.proj()))
print(len(ta3p100.DendriteWidth() & "min_width_max>0" & ta3p100.CompartmentOrphan.ComponentOrphan.proj() & "compartment_type='Dendrite'"))
print(len(ta3p100.DendriteWidth() & "min_width_max>0" & ta3p100.CompartmentOrphan.ComponentOrphan.proj() & "compartment_type='Dendrite'" & component_total))



In [ ]:
dendrite_accepted = (ta3p100.DendriteWidth() & "min_width_max>0" & ta3p100.CompartmentOrphan.ComponentOrphan.proj() & "compartment_type='Dendrite'" & component_total)
dendrite_accepted

In [ ]:
percentages_to_fetch = ["min_width_perc_50","min_width_perc_55","min_width_perc_60",
                      "min_width_perc_65","min_width_perc_70","min_width_perc_75",
                       "min_width_perc_80","min_width_perc_85","min_width_perc_90",
                       "min_width_perc_95","min_width_max"]
min_width_percentiles = dendrite_accepted.fetch("min_width_perc_50","min_width_perc_55","min_width_perc_60",
                      "min_width_perc_65","min_width_perc_70","min_width_perc_75",
                       "min_width_perc_80","min_width_perc_85","min_width_perc_90",
                       "min_width_perc_95","min_width_max")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# plt.figure(i)
#     sns.countplot(x=col, data=df1)

for i,perc in enumerate(min_width_percentiles):
    plt.figure(i)
    plt.title(percentages_to_fetch[i])
    sns.distplot(perc)

In [ ]:
#finding neurites to show
orphan_300 = (dendrite_accepted & "min_width_perc_80>295" & "min_width_perc_80<300").fetch("segment_id")[0]
orphan_450 = (dendrite_accepted & "min_width_perc_80>445" & "min_width_perc_80<455").fetch("segment_id")[0]
orphan_600 = (dendrite_accepted & "min_width_perc_80>595" & "min_width_perc_80<605").fetch("segment_id")[:5]
print(orphan_300)
print(orphan_450)
print(orphan_600)

In [ ]:
#finding neurites to show
orphan_300 = (dendrite_accepted & "min_width_perc_95>295" & "min_width_perc_95<300").fetch("segment_id")[0:2]
orphan_450 = (dendrite_accepted & "min_width_perc_95>495" & "min_width_perc_95<505").fetch("segment_id")[0:2]
orphan_600 = (dendrite_accepted & "min_width_perc_95>595" & "min_width_perc_95<605").fetch("segment_id")[:5]
print(orphan_300)
print(orphan_450)
print(orphan_600)

In [ ]:
#finding neurites to show
orphan_300 = (dendrite_accepted & "min_width_perc_95>295" & "min_width_perc_80<300").fetch("segment_id")[0:2]
orphan_450 = (dendrite_accepted & "min_width_perc_90>445" & "min_width_perc_80<455").fetch("segment_id")[0:2]
orphan_600 = (dendrite_accepted & "min_width_perc_90>595" & "min_width_perc_80<605").fetch("segment_id")[:5]
print(orphan_300)
print(orphan_450)
print(orphan_600)

# Jake wants to do the stark difference of 0.1, 0.5 and 0.9 percentile parts

In [ ]:
perc_95_data = dendrite_accepted.fetch("min_width_perc_95")
sns.distplot(perc_95_data)


In [ ]:
#find the percentiles
percentile_numbers = [10,50,90]

percentile_list= []
for pn in percentile_numbers:
    percentile_list.append(np.percentile(perc_95_data,pn))
# percentile_list = [np.percentile(perc_95_data,10),
#                    np.percentile(perc_95_data,50),
#                   np.percentile(perc_95_data,90),
#                   ]

print(percentile_list)

In [ ]:
for i,perc in enumerate(percentile_list):
    print("Segments for Percentile: "+str(percentile_numbers[i]))
    orphan_segemnts = (dendrite_accepted & "min_width_perc_95>" + str(perc-5) & "min_width_perc_95<"+ str(perc+5)).fetch("segment_id")[0:5]
    print(orphan_segemnts)

# make Official looking graph for 95th percentile

In [ ]:
dendrite_accepted.fetch("min_width_perc_95")

In [ ]:
import seaborn as sns
sns.set()

ax =  sns.distplot(dendrite_accepted.fetch("min_width_perc_95"),kde=False)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.tick_params(axis='y', which='major', labelsize=14)
ax.set_ylabel("Count",fontdict=dict(fontsize=20))
ax.set_xlabel("dendrite width (nm)",fontdict=dict(fontsize=20))
ax.set_title("Orphan Dendrite Width Distribution",fontdict=dict(fontsize=20))
ax.set

In [ ]:
import seaborn as sns
sns.set()

ax =  sns.distplot(ta3p100.SignificantOrphanDendriteWidthLength.fetch("skeleton_length")/1000,kde=False)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.tick_params(axis='y', which='major', labelsize=14)
ax.set_ylabel("Count",fontdict=dict(fontsize=20))
ax.set_xlabel("dendrite length (microns)",fontdict=dict(fontsize=20))
ax.set_title("Orphan Dendrite Length Distribution",fontdict=dict(fontsize=20))
ax.set

# Redoing the Plot in the same manner as Stelios

In [ ]:
help(plt.xticks)

In [ ]:


fig, axs = plt.subplots(1, len(params), figsize=(n_axrow*scale,n_axcol*scale))

In [ ]:
import matplotlib.pyplot as plt
plt.hist(ta3p100.SignificantOrphanDendriteWidthLength.fetch("skeleton_length")/1000,bins=50,color='k') 
plt.xlabel("dendrite length (microns)", fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.title("Orphan Dendrite Length Distribution",fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid(b=None)
plt.
sns.despine()


In [ ]:
"""
import seaborn as sns
sns.set()

ax =  sns.distplot(dendrite_accepted.fetch("min_width_perc_95"),kde=False)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.tick_params(axis='y', which='major', labelsize=14)
ax.set_ylabel("Count",fontdict=dict(fontsize=20))
ax.set_xlabel("dendrite width (nm)",fontdict=dict(fontsize=20))
ax.set_title("Orphan Dendrite Width Distribution",fontdict=dict(fontsize=20))
ax.set


"""

import matplotlib.pyplot as plt
plt.hist(ta3p100.SignificantOrphanDendriteWidthLength.fetch("width")/1000,bins=50,color='k') 
plt.xlabel("dendrite width (nm)", fontsize=15)
plt.ylabel('Count', fontsize=15)
plt.title("Orphan Dendrite Width Distribution",fontsize=15)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.grid(b=None)
sns.despine()


In [ ]:
print(plt.__version__)

In [ ]:
params = [ta3p100.SignificantOrphanDendriteWidthLength.fetch("width"),ta3p100.SignificantOrphanDendriteWidthLength.fetch("skeleton_length")/1000]
scale=2
n_axrow = len(params)
n_axcol = 1
fig, axs = plt.subplots(1, len(params), figsize=(n_axrow*scale,n_axcol*scale))
for ax, param, title in zip(axs, params, titles):
    ax.hist(param, bins=50, color='k')
    ax.set_xlabel(title, fontsize=15)
axs[0].set_ylabel('Number of Axons', fontsize=15)
fig.suptitle('Intrinsic Orphan Axon Properties', fontsize=25, y=1.02)
# sns.despine()
fig.set_dpi(150)

In [ ]:
"""
************ CODE THAT FINDS THE 
"""

In [ ]:
min_width_perc_50 = np.percentile(widths, 50),
min_width_perc_55 = np.percentile(widths, 55),
min_width_perc_60 = np.percentile(widths, 60),
min_width_perc_65 = np.percentile(widths, 65),
min_width_perc_70 = np.percentile(widths, 70),
min_width_perc_75 = np.percentile(widths, 75),
min_width_perc_80 = np.percentile(widths, 80),
min_width_perc_85 = np.percentile(widths, 85),
min_width_perc_90 = np.percentile(widths, 90),
min_width_perc_95 = np.percentile(widths, 95),
min_width_max = np.max(widths)

print(min_width_perc_50)
print(min_width_perc_55)
print(min_width_perc_60)
print(min_width_perc_65)
print(min_width_perc_70)
print(min_width_perc_75)
print(min_width_perc_80)
print(min_width_perc_85)
print(min_width_perc_90)
print(min_width_perc_95)
print(min_width_max)

In [ ]:
print(widths.shape)
np.max(widths[widths<1000])

In [ ]:
import matplotlib.pyplot as plt

widths = ta3p100.SignificantOrphanDendriteWidthLength.fetch("width")
lengths = ta3p100.SignificantOrphanDendriteWidthLength.fetch("skeleton_length")/1000


In [ ]:
lengths = lengths[widths < 1000]
widths = widths[widths < 1000]


fig, (ax1, ax2) = plt.subplots(1,2)

ax1.hist(widths, bins=50, color='k')
ax2.hist(lengths, bins=50, color='k')

ax1.tick_params(labelsize=12)
ax2.tick_params(labelsize=12)
ax1.set_xlim([200,1000])
ax1.set_xlabel('dendrite width ($\mu m$)', fontsize=12)
ax2.set_xlabel('dendrite length (per 100 $\mu m$)', fontsize=12)

ax1.set_ylabel('Number of Orphan Dendrites', fontsize=12)
fig.set_size_inches(8,3)
fig.set_dpi(300)

#plt.grid(False)
ax1.grid(b=None)
ax2.grid(b=None)
sns.despine()

In [ ]:
help(ax.set_ylabel)

fontdict=dict(fontsize=13)

In [ ]:
perc_95_data = dendrite_accepted.fetch("min_width_perc_95")

import matplotlib.pyplot as plt


plt.hist(perc_95_data,color = '#2977E7',label='shaft')
# plt.plot([0,len(x)],[0.5,0.5],color="#322FE7",label='shaft_threshold')
# plt.plot(range(0,len(y)),y,color="#E7803C",label='spine head')
# plt.plot([0,len(x)],[0.6,0.6],color="#E73035",label='spine_head_threshold')

# plt.ylabel('proportion',fontsize=18)
# plt.xlabel('neurite index (spine head sorted)',fontsize=18)
# plt.legend(loc=3)


# plt.tick_params(axis='x', which='major', labelsize=13)
# plt.tick_params(axis='y', which='major', labelsize=18)

plt.show()


In [ ]:
dendrite_labels = ["Apical","Basal","Oblique","Dendrite"]


#dj.U("segment_id").aggr(ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels]
#ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500"
exhitatory = dj.U("segment_id").aggr(ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")





In [ ]:
dendrite_labels = ["Apical","Basal","Oblique","Dendrite"]


#dj.U("segment_id").aggr(ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels]
#ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500"
exhitatory = dj.U("segment_id").aggr(ta3p100.CompartmentFinal.ComponentFinal() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

orphan = dj.U("segment_id").aggr(ta3p100.CompartmentOrphan.ComponentOrphan() & [dict(compartment_type=x) for x in dendrite_labels] & "n_vertex_indices>500",
                       dendrite_vertex_count="sum(n_vertex_indices)")

restriction_threshold = 15000
dendrite_restriction = ((exhitatory & "dendrite_vertex_count>" + str(restriction_threshold)).proj() + 
                            (orphan & "dendrite_vertex_count>" + str(restriction_threshold)).proj()) - ta3p100.SegmentExclude.proj()

#dendrite_restriction



# FINDING THE NUMBERS FOR THE FINAL REPORT

In [ ]:
"""
Orphaned dendritic segments ranged in total length from XX um to XX um,
 Maximum (95th percentile) dendritic width varied from XX um to XXum
 Synapse density ranged from XX synapses/100 um to XX, with densities 
[higher/lower] on L5 trunks compared to the small and medium-diameter dendrites.
"""


In [ ]:
#searching for the widths
fat_orphan_dendrites = (ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'" & 
        (ta3p100.SignificantOrphanDendriteWidthLength() & "width > 600"))

In [ ]:
#Orphaned dendritic segments ranged in total length from XX um to XX um,
#Maximum (95th percentile) dendritic width varied from XX um to XXum
skeleton_length = ta3p100.SignificantOrphanDendriteWidthLength.fetch("skeleton_length")
print("min(skeleton_length) = " + str(min(skeleton_length/1000)))
print("max(skeleton_length) = " + str(max(skeleton_length/1000)))

In [ ]:
#Maximum (95th percentile) dendritic width varied from XX um to XXum
width_data = ta3p100.SignificantOrphanDendriteWidthLength.fetch("width")
print("min(width_data) = " + str(min(width_data)))
print("max(width_data) = " + str(max(width_data)))

In [ ]:
#table that aggregates number of synapses over postsyn
postsyn_n_synapses = dj.U("postsyn").aggr(ta3p100.Synapse,n_synapses="count(*)")
postsyn_data = postsyn_n_synapses.fetch("postsyn","n_synapses")

In [ ]:
postsyn = postsyn_data[0]
n_synapses = postsyn_data[1]

n_synapses_lookup = dict()
for i,p in enumerate(postsyn):
    n_synapses_lookup[str(p)] = n_synapses[i]


In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
"""
Synapse density ranged from XX synapses/100 um to XX, with densities 
[higher/lower] on L5 trunks compared to the small and medium-diameter dendrites.
"""
l5_width_threshold = 700

# l5_apicals = (ta3p100.SignificantOrphanDendriteWidthLength & "width>="+str(l5_width_threshold) )
# others = (ta3p100.SignificantOrphanDendriteWidthLength & "width<"+str(l5_width_threshold) )

#get widths and synaptic density and see if there is difference
orphan_data = ta3p100.SignificantOrphanDendriteWidthLength.fetch("segment_id","width","skeleton_length")
segment_id = orphan_data[0]
orphan_width = orphan_data[1]
skeleton_length = orphan_data[2]

orphan_synapse_density = []
for i,seg_id in tqdm(enumerate(segment_id)):
    key = dict(postsyn=seg_id)
    #get the number of synapses
    n_syn = n_synapses_lookup[str(seg_id)]
    
    #get the length of the dendrite
    length_nm = skeleton_length[i]

    #calculate the synapse density
    density = n_syn/(length_nm*1000*100)
    
    orphan_synapse_density.append(density*1000000000)
    
print(min(orphan_synapse_density))
print(max(orphan_synapse_density))




# l5_synapse_density = []
# other_synapse_density = []

# for l5 in l5_apicals:
#     key = dict(postsyn=l5)
#     #get the number of synapses
#     n_syn = (postsyn_n_synapses & key).fetch1("n_synapses")
    
#     #get the length of the dendrite
#     length_nm = (ta3p100.SignificantOrphanDendriteWidthLength & "segment_id=" + str(l5)).fetch1("skeleton_length")
    
#     #calculate the synapse density
#     density = n_syn/(length_nm*1000*100)
    
#     l5_synapse_density.append(density)

# for l5 in others:
#     key = dict(postsyn=l5)
#     #get the number of synapses
#     n_syn = (postsyn_n_synapses & key).fetch1("n_synapses")
    
#     #get the length of the dendrite
#     length_nm = (ta3p100.SignificantOrphanDendriteWidthLength & "segment_id=" + str(l5)).fetch1("skeleton_length")
    
#     #calculate the synapse density
#     density = n_syn/(length_nm*1000*100)
    
#     l5_synapse_density.append(density)
    
    


In [ ]:
print(min(orphan_synapse_density))
print(max(orphan_synapse_density))

In [ ]:
#plot the distribution
plt.scatter(x = orphan_width,y = orphan_synapse_density)

ax = sns.scatterplot(orphan_width,orphan_synapse_density)
ax.tick_params(axis='x', which='major', labelsize=15)
ax.tick_params(axis='y', which='major', labelsize=14)
ax.set_ylabel("synaptic density \nsyn/100 microns",fontdict=dict(fontsize=20))
ax.set_xlabel("Width of orphan (nm)",fontdict=dict(fontsize=20))
ax.set_title("Orphan Dendrite Synapse Density vs. Width",fontdict=dict(fontsize=20))
ax.set
# plt.xlabel("width of orphan (nm)")
# plt.show()
    

In [ ]:
l5_width_threshold = 700


l5_orphan_data = (ta3p100.SignificantOrphanDendriteWidthLength & 
               "width>="+str(l5_width_threshold)).fetch("segment_id","width","skeleton_length")
l5_segment_id = l5_orphan_data[0]
l5_orphan_width = l5_orphan_data[1]
l5_skeleton_length = l5_orphan_data[2]

orphan_data = (ta3p100.SignificantOrphanDendriteWidthLength & 
               "width<"+str(l5_width_threshold)).fetch("segment_id","width","skeleton_length")
segment_id = orphan_data[0]
orphan_width = orphan_data[1]
skeleton_length = orphan_data[2]


l5_synapse_density = []
other_synapse_density = []

for i,seg_id in tqdm(enumerate(l5_segment_id)):
    key = dict(postsyn=seg_id)
    #get the number of synapses
    n_syn = n_synapses_lookup[str(seg_id)]
    
    #get the length of the dendrite
    length_nm = l5_skeleton_length[i]

    #calculate the synapse density
    density = n_syn/(length_nm*1000*100)
    
    l5_synapse_density.append(density*1000000000)
    
for i,seg_id in tqdm(enumerate(segment_id)):
    key = dict(postsyn=seg_id)
    #get the number of synapses
    n_syn = n_synapses_lookup[str(seg_id)]
    
    #get the length of the dendrite
    length_nm = skeleton_length[i]

    #calculate the synapse density
    density = n_syn/(length_nm*1000*100)
    
    other_synapse_density.append(density*1000000000)

In [ ]:
import math
print("L5 Synapse Density Statistics (Width >=700 nm )")
print("count = " + str(len(l5_synapse_density)))
print("max synapse density = " + str(max(l5_synapse_density)))
print("mean synapse density = " + str(np.mean(l5_synapse_density)))
print("median synapse density = " + str(np.median(l5_synapse_density)))
print("min synapse density = " + str(min(l5_synapse_density)))
print("\n\n")
print("Non L5 Synapse Density Statistics (Width <700 nm )")
print("count = " + str(len(other_synapse_density)))
print("max synapse density = " + str(max(other_synapse_density)))
print("mean synapse density = " + str(np.mean(other_synapse_density)))
print("median synapse density = " + str(np.median(other_synapse_density)))
print("min synapse density = " + str(min(other_synapse_density)))

In [ ]:
l5_synapse_density
key = dict(postsyn=648518346345770568)
#get the number of synapses
n_syn = (postsyn_n_synapses & key).fetch1("n_synapses")

#get the length of the dendrite
length_nm = (ta3p100.SignificantOrphanDendriteWidthLength & "segment_id=" + str(648518346345770568)).fetch1("skeleton_length")

#calculate the synapse density
density = n_syn/(length_nm*1000*100)

l5_synapse_density.append(density)

In [ ]:
(ta3p100.Synapse() & "postsyn=648518346341379951") - ta3p100.SynapseExcludeFinal

In [ ]:
sizes = (ta3p100.CompartmentFinal.ComponentFinal & "compartment_type='Apical'").fetch("n_vertex_indices")
max(sizes)

